About the Dataset:
1. id: unique id for a news article
2. title: the title of the news article
3. author: author of the news article
4. text: the text of the article, could be incomplete
5. label: a label that marks wheter the news article is real or fake
```
1: Fake news
0: Real News
```

Import the Dependencies

In [4]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\malub\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

## Stopwords
Stopwords are common words in english that usually don't carry much meaning to tasks of NLP.

In [9]:
stopwords.words('English') # Printing the stopwords

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

## Data Pre-Processing

In [10]:
# Load the dataset into a pandas DataFrame
news_data = pd.read_csv('news.csv')

In [11]:
news_data.shape

(6335, 4)

In [12]:
news_data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [15]:
# Counting the number os missing values in the dataset
news_data.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [17]:
news_data = news_data.drop(columns=["Unnamed: 0"])

In [18]:
news_data["content"] = news_data["title"] + " " + news_data["text"]

0       You Can Smell Hillary’s Fear Daniel Greenfield...
1       Watch The Exact Moment Paul Ryan Committed Pol...
2       Kerry to go to Paris in gesture of sympathy U....
3       Bernie supporters on Twitter erupt in anger ag...
4       The Battle of New York: Why This Primary Matte...
                              ...                        
6330    State Department says it can't find emails fro...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332    Anti-Trump Protesters Are Tools of the Oligarc...
6333    In Ethiopia, Obama seeks progress on peace, se...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: content, Length: 6335, dtype: object

In [21]:
# Separating the data & label
X = news_data.drop("label", axis=1)
y = news_data["label"]

In [22]:
X, y

(                                                  title  \
 0                          You Can Smell Hillary’s Fear   
 1     Watch The Exact Moment Paul Ryan Committed Pol...   
 2           Kerry to go to Paris in gesture of sympathy   
 3     Bernie supporters on Twitter erupt in anger ag...   
 4      The Battle of New York: Why This Primary Matters   
 ...                                                 ...   
 6330  State Department says it can't find emails fro...   
 6331  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
 6332  Anti-Trump Protesters Are Tools of the Oligarc...   
 6333  In Ethiopia, Obama seeks progress on peace, se...   
 6334  Jeb Bush Is Suddenly Attacking Trump. Here's W...   
 
                                                    text  \
 0     Daniel Greenfield, a Shillman Journalism Fello...   
 1     Google Pinterest Digg Linkedin Reddit Stumbleu...   
 2     U.S. Secretary of State John F. Kerry said Mon...   
 3     — Kaydee King (@KaydeeKing) Nov

## Stemming:
Stemming is the process of reducing a word to its root word

Example:
actor, actress, acting --> act

In [23]:
port_stem = PorterStemmer()

In [24]:
def stemming(content):
    # Remove caracters that aren't letters
    content = re.sub('[^a-zA-Z]', ' ', content)
    # Transform into lower and split into words
    words = content.lower().split()
    # Apply stemmer and remove stopwords
    words = [port_stem.stem(word) for word in words if word not in stopwords.words('english')]
    # Join the words together into a string
    return ' '.join(words)
    

In [25]:
news_data['content'] = news_data['content'].apply(stemming)

In [26]:
X = news_data['content']
Y = news_data['label']

In [27]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

## Splitting the Dataset into Train and Test Data

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## Training the Model: Logistic Regression
Despite its name, Logistic Regression is used for classification, usually binary classification. In this case:
- Class 1 → FAKE
- Class 0 → REAL

The model computes a linear combination of features
z = β0​ + β1​X1​ + β2​X2 ​+⋯+ βn​Xn​
- X_i → feature i (TF-IDF of a word)
- β_i → weight learned by the model

Then it passes this linear combination through the sigmoid function
σ(z)=1+e−z1​
The output is a value between 0 and 1, interpreted as the probability that the news is FAKE.

The closest to 1, the fakiest.

In [32]:
model = LogisticRegression()

In [33]:
model.fit(X_train, Y_train)

LogisticRegression()

## Evaluation

In [34]:
# Accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
training_data_accuracy

0.9532359905288083

In [35]:
# Accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
test_data_accuracy

0.9163378058405682

## Making a Predictive System

In [37]:
X_new = X_test[0]

prediction = model.predict(X_new)

if prediction[0] == 'REAL':
    print('The news are Real')
else:
    print('The news are Fake')

The news are Fake


In [39]:
Y_test.iloc[0]

'FAKE'